In [ ]:
#Getting all the necessary imports
from fastai import *
#imports all the necessary pieces from the library. fastai is designed to be usable in a read–eval–print loop(REPL)
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#This module provides a portable way of using operating system dependent functionality
import os
import scipy as sp
from functools import partial
from sklearn import metrics
#to get the count of elements using counter
from collections import Counter
#this is for all learner.vision
from fastai.callbacks import *

In [ ]:
#Import libraries
import cv2
import os
#Function to extract frames
#directory path, where my video images will be stored
mydir = r'../input/datajpg'
try:
    if not os.path.exists(os.path.dirname(mydir)):
        os.makedirs(os.path.dirname(mydir))
except OSError as err:
    print(err)
#Capture vidoe from video file
cap = cv2.VideoCapture(r"../input/datajpg/mery.mp4.mp4")
#Counter Variable
count = 0

while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break
    
if ret == True:
    print('Read %d frame: ' % count, ret)
# save frame as JPEG file
cv2.imwrite(os.path.join(r'../input/datajpg', "frame{:d}.jpg".format(count)), frame)
count += 1

#cv2.destroyAllWindows()   
# When everything done, release the capture
cap.release()


In [ ]:
##the line which gives the path of  data set
#retreving captured frames from local file path
data_folder = Path("../input/my-nsfw-dataset")
#If you want to quickly get a set of random transforms that have worked well in a wide range of tasks, you should use the get_transforms function. The most important parameters to adjust are do_flip and flip_vert, depending on the type of images you have.
#get_transforms returns a tuple of two lists of transforms: one for the training set and one for the validation set 
trfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)

In [ ]:
# I can tell (for JPEGs) PIL only accepts it as a valid JPEG Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#ImageDataBunch from_* methods are used, the size and resize_method 
data = (ImageList.from_folder(data_folder)
        .split_by_rand_pct(0.1)
        .label_from_folder()
        .add_test_folder("../input/my-nsfw-dataset")
        .transform(trfms, size=128)
        .databunch(bs=64, device= torch.device('cpu'))
        .normalize())

In [ ]:
#computer vision learner
#vision.learner is the module that defines the cnn_learner method, to easily get a model suitable for transfer learning.
#os.getcwd This method returns current working directory of a process
learn = cnn_learner(data, models.resnet101, metrics=[error_rate, accuracy], model_dir = os.getcwd())

In [ ]:
#Explore lr from start_lr to end_lr over num_it iterations in learn. If stop_div, stops when loss diverges.
learn.lr_find(stop_div=False, num_it=200)

In [ ]:
#plot the loss vs learning rage
learn.recorder.plot(suggestion = True)
# the point which gives minimum loss at particular learning point
min_grad_lr = learn.recorder.min_grad_lr

In [ ]:
lr = 1e-03
# no of ephocs to produce accuracy,loss and error rate 
learn.fit_one_cycle(50, slice(lr))

In [ ]:
learn.path = Path()

In [ ]:
#ls - list the contents of the current directory
ls